In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../")
import logging 
logging.basicConfig(level=logging.INFO)

In [2]:
import shelve
from pathlib import Path
import wikilanguage
import itertools
import pipelines
from wikidata_parser import WikiDataParser
from wikidata_parser import WikiDataInheritanceGraph
data_path = Path("/mnt/evo/projects/wikilanguage/data/20200701")
wikidata_path = data_path / "wikidata-20200706-all.json.gz"
old_wikidata_path = Path("/mnt/evo/projects/wikilanguage/data/20200101/wikidata-20200113-all.json.gz")

In [3]:
%time enwiki_shelve = shelve.open("/home/tdimson/projects/wikilanguage/working-dir-20200701/enwiki")

CPU times: user 1min 26s, sys: 1.32 s, total: 1min 27s
Wall time: 1min 27s


In [ ]:
enwiki_old_shelve = shelve.open("/mnt/evo/projects/wikilanguage/data/working-dir/enwiki")

In [4]:
%time parent_finder = WikiDataInheritanceGraph.load("/home/tdimson/projects/wikilanguage/working-dir-20200701/inheritance.pickle").parent_finder()

CPU times: user 1min 32s, sys: 11.8 s, total: 1min 44s
Wall time: 1min 44s


In [5]:
%time alias_map = pipelines.build_alias_map(enwiki_shelve)

CPU times: user 9min 51s, sys: 34.1 s, total: 10min 26s
Wall time: 10min 29s


In [ ]:
def print_stats(wikidata_path, shelve, wiki_name, alias_map, sample_size=2000):
    num_no_title = 0
    num_empty = 0
    num_considered = 0
    num_missing_article = 0
    num_aliases = 0
    for line in itertools.islice(pipelines.buffered_lines_with_progress(wikidata_path), sample_size):
        num_considered += 1
        entry = WikiDataParser.parse_dump_line(line, whitelisted_wikis={wiki_name})
        if not entry:
            num_empty += 1
            continue
            
        wiki_title = entry.titles_by_wiki.get(wiki_name)
        if not wiki_title:
            num_no_title += 1
            continue
            
        if wiki_title in alias_map:
            wiki_title = alias_map[wiki_title]
            num_aliases += 1
        
        if not shelve.get(wiki_title):
            if num_missing_article % 100 == 0:
                print(f"Sample missing article for {entry.id}: '{wiki_title}'")
            num_missing_article += 1
            continue
            
    
    print(f"""
Considered: {num_considered}
Empty: {(100 * num_empty / num_considered):.2f}%
{wiki_name} Aliases: {100 * num_aliases / num_considered:.2f}%
{wiki_name} Missing Title: {100 * num_no_title / num_considered:.2f}%
{wiki_name} Missing Article: {100 * num_missing_article / num_considered:.2f}%
    """.strip())

In [ ]:
print_stats(wikidata_path, enwiki_shelve, "enwiki", alias_new, sample_size=1_000_000)

In [7]:
nm = pipelines.write_full_wiki_csv(
    wikidata_path=str(wikidata_path),
    output_path=str(data_path / "enwiki.tsv"),
    #output_path="test.tsv",wwwwwwww
    article_shelf=enwiki_shelve,
    wiki_name="enwiki",
    parent_finder=parent_finder,
    alias_map=alias_map,
    #limit=50000,
)

INFO:root:Writing TSV


WARN: Unexpected date in 'Sarashina Nikki' line Q217955: {'mainsnak': {'snaktype': 'somevalue', 'property': 'P577', 'datatype': 'time'}, 'type': 'statement', 'id': 'Q217955$59378273-44d7-f4dc-b465-d868274e2c43', 'rank': 'normal'}
WARN: Unexpected date in 'StarCraft: Ghost' line Q605211: {'mainsnak': {'snaktype': 'novalue', 'property': 'P577', 'datatype': 'time'}, 'type': 'statement', 'id': 'Q605211$16d4b38d-4849-e28d-c13e-d52087d1a986', 'rank': 'normal'}
WARN: Unexpected date in 'Book on Numbers and Computation' line Q848171: {'mainsnak': {'snaktype': 'somevalue', 'property': 'P577', 'datatype': 'time'}, 'type': 'statement', 'qualifiers': {'P1319': [{'snaktype': 'value', 'property': 'P1319', 'hash': 'bda5cb13f4da62b38206b6e8fd1955881b2fb1ec', 'datavalue': {'value': {'time': '-0186-00-00T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 9, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985786'}, 'type': 'time'}, 'datatype': 'time'}]}, 'qualifiers-order': ['P1319'], 'i